In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import numpy as np
import gzip
import os
import anndata as ad
import scanpy as sc
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [10]:
sc.settings.verbosity = 0
pd.set_option('display.max_columns', 50)
#pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 100)
sc.settings.n_jobs=10

In [11]:
adata_comb = sc.read_h5ad("/fs/cbsuvlaminck5/workdir/sj657/lymph_node/for_public/output_files/objects/adata_comb_allTP_whole_meta_raw.h5ad")

In [12]:
adata_comb.obs['num_clones_TR'] = adata_comb.obs[['num_clones_TRA','num_clones_TRB']].sum(axis=1)
adata_comb.obs['num_clones_IG'] = adata_comb.obs[['num_clones_IGK','num_clones_IGL','num_clones_IGH']].sum(axis=1)
# convert 0 to na?
adata_comb.obs[['num_clones_TR','num_clones_IG']] = adata_comb.obs[['num_clones_TR','num_clones_IG']].replace(0.0, np.nan)
adata_comb.obs.index = adata_comb.obs['Index']

# Figure 1b: Spatial Map on Refine Cell types

In [ ]:
ref_color_dict = {"B_plasma": "#AE05D2", #purple
              "B_GC": "#FFA64B", # like pink
              "B_Mem": "#0028FF", #deep blue
              "B_Naive": "#008DFF", #light green
              "B_Act_Naive": "#8769D0", #light green
              "T_CD4_Cycling": "#006745", #green
              "T_CD4": "#00A175",
              "T_CD8_Cycling": "#9BB0A5",
              "T_CD8": "#7A7B1B",
              "NK_ILC":"#5B5295",
              "Macrophage": "#DF1F1F",
              "Monocytes":"#C24F76",
              "DCs": "#FF6F48",
              "FDC":"#000000",
              "VSMC": "#7E5500",
              "Endo": "#843200",
              "Adipocytes": "#792E57"
             }
# Font properties for scale bar
fontprops = fm.FontProperties(size=8)
sc.settings.set_figure_params(dpi=500, dpi_save= 500, fontsize=2, facecolor='white', frameon=False, figsize=(3, 3), vector_friendly = False, transparent=True, format="pdf")
for tp in ["Mock","D3PI","D7PI","D10PI","D14PI","D21PI"]:
    print(tp)
    # Add Padding
    global_min_x, global_max_x, global_min_y, global_max_y = float('inf'), float('-inf'), float('inf'), float('-inf')
    adata = adata_comb[adata_comb.obs['Sample']==tp].copy()
    coords = adata.obsm['spatial']
    global_min_x, global_max_x = min(global_min_x, coords[:, 0].min()), max(global_max_x, coords[:, 0].max())
    global_min_y, global_max_y = min(global_min_y, coords[:, 1].min()), max(global_max_y, coords[:, 1].max())

    # Optional padding
    padding = 50
    global_min_x -= padding
    global_max_x += padding
    global_min_y -= padding
    global_max_y += padding
    

    # Try add scale bar
    fig, ax = plt.subplots(1,1, figsize = (3,3))
    g = "Refine_celltypes"
    sc.pl.spatial(adata, color = g,title='',spot_size=35,show=False,legend_loc=None,palette=ref_color_dict, ax = ax)
    # Set consistent axes limits
    ax.set_xlim(global_min_x, global_max_x)
    ax.set_ylim(global_max_y, global_min_y)
    ax.set_aspect('equal', adjustable='box')

    # add the scale bar
    scalebar = AnchoredSizeBar(ax.transData, 1.54*500, '', 'lower left',
                                pad=0.5, color='black', frameon=False, size_vertical=0.5,
                                fontproperties=fontprops)
    ax.add_artist(scalebar)
    # save
    plt.savefig('figures/sp_'+tp+"_"+g+'.png',dpi = 500,transparent=True)
    plt.savefig('figures/sp_'+tp+"_"+g+'.pdf',dpi = 500,transparent=True)

# Figure 1c: Spatial Map on TCR & BCR clones

In [ ]:
from matplotlib import colors
# Reds for TR
colors2 = plt.cm.Reds(np.linspace(0, 1, 128))
colorsComb = np.vstack([colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)
my_cmap = mymap(np.arange(mymap.N))
my_cmap[:,-1] = np.linspace(0, 1, mymap.N)
my_cmap = colors.ListedColormap(my_cmap)
# Blues for IG
colors2 = plt.cm.Blues(np.linspace(0, 1, 128))
colorsComb = np.vstack([colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)
my_cmap2 = mymap(np.arange(mymap.N))
my_cmap2[:,-1] = np.linspace(0, 1, mymap.N)
my_cmap2 = colors.ListedColormap(my_cmap2)

In [ ]:
adata_comb_na = adata_comb.copy()
adata_comb_na.obs[['num_clones_TR','num_clones_IG']] = np.nan
adata_comb_fillna = adata_comb.copy()
adata_comb_fillna.obs[['num_clones_TR','num_clones_IG']] = adata_comb_fillna.obs[['num_clones_TR','num_clones_IG']].replace(np.nan, 0.0)

In [ ]:
# plot two varaibles in the same figure
g = "num_clones_IG";i = "num_clones_TR"
vmax_g = np.nanpercentile(adata_comb.obs[g],99) # too many zero
vmax_i = 4
sc.settings.set_figure_params(dpi=500, dpi_save= 500, fontsize=12, facecolor='white', frameon=False, figsize=(2, 2), vector_friendly = False, transparent=True, format="pdf")
fig, axs = plt.subplots(1,6, figsize = (14,4))
plt.subplots_adjust(wspace=0)
# should I plot a layer of na
f1 = sc.pl.spatial(adata_comb_na[adata_comb_na.obs['Sample']=="Mock"], color = g,spot_size=60,  ax = axs[0],show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_g,na_color="#E2E2E2") # make an empty layer with the shape
f1 = sc.pl.spatial(adata_comb_fillna[adata_comb_fillna.obs['Sample']=="Mock"], color = g,spot_size=60,  ax = axs[0],show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_g,cmap=my_cmap2) # blue
f1 = sc.pl.spatial(adata_comb_fillna[adata_comb_fillna.obs['Sample']=="Mock"], color = i,spot_size=60,  ax = axs[0],show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_i,cmap=my_cmap) # Red
f1 = sc.pl.spatial(adata_comb_na[adata_comb_na.obs['Sample']=="D3PI"], color = g,spot_size=60,  ax = axs[1],show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_g,na_color="#E2E2E2")
f1 = sc.pl.spatial(adata_comb_fillna[adata_comb_fillna.obs['Sample']=="D3PI"], color = g,spot_size=60,  ax = axs[1],show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_g,cmap=my_cmap2)
f1 = sc.pl.spatial(adata_comb_fillna[adata_comb_fillna.obs['Sample']=="D3PI"], color = i,spot_size=60,  ax = axs[1],show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_i,cmap=my_cmap)
f1 = sc.pl.spatial(adata_comb_na[adata_comb_na.obs['Sample']=="D7PI"], color = g,spot_size=60,  ax = axs[2],show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_g,na_color="#E2E2E2")
f1 = sc.pl.spatial(adata_comb_fillna[adata_comb_fillna.obs['Sample']=="D7PI"], color = g,spot_size=60,  ax = axs[2],show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_g,cmap=my_cmap2)
f1 = sc.pl.spatial(adata_comb_fillna[adata_comb_fillna.obs['Sample']=="D7PI"], color = i,spot_size=60,  ax = axs[2],show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_i,cmap=my_cmap)
f1 = sc.pl.spatial(adata_comb_na[adata_comb_na.obs['Sample']=="D10PI"], color = g,spot_size=60,  ax = axs[3],show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_g,na_color="#E2E2E2")
f1 = sc.pl.spatial(adata_comb_fillna[adata_comb_fillna.obs['Sample']=="D10PI"], color = g,spot_size=60,  ax = axs[3],show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_g,cmap=my_cmap2)
f1 = sc.pl.spatial(adata_comb_fillna[adata_comb_fillna.obs['Sample']=="D10PI"], color = i,spot_size=60,  ax = axs[3],show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_i,cmap=my_cmap)
f1 = sc.pl.spatial(adata_comb_na[adata_comb_na.obs['Sample']=="D14PI"], color = g,spot_size=60,  ax = axs[4],show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_g,na_color="#E2E2E2")
f1 = sc.pl.spatial(adata_comb_fillna[adata_comb_fillna.obs['Sample']=="D14PI"], color = g,spot_size=60,  ax = axs[4],show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_g,cmap=my_cmap2)
f1 = sc.pl.spatial(adata_comb_fillna[adata_comb_fillna.obs['Sample']=="D14PI"], color = i,spot_size=60,  ax = axs[4],show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_i,cmap=my_cmap)
f1 = sc.pl.spatial(adata_comb_na[adata_comb_na.obs['Sample']=="D21PI"], color = g,spot_size=60,  ax = axs[5],show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_g,na_color="#E2E2E2")
f1 = sc.pl.spatial(adata_comb_fillna[adata_comb_fillna.obs['Sample']=="D21PI"], color = g,spot_size=60,  ax = axs[5],show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_g,cmap=my_cmap2)
f1 = sc.pl.spatial(adata_comb_fillna[adata_comb_fillna.obs['Sample']=="D21PI"], color = i,spot_size=60,  ax = axs[5],show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_i,cmap=my_cmap)
plt.savefig('figures/sp_IGTR_allTP.png',dpi = 500,transparent=True)
plt.savefig('figures/sp_IGTR_allTP.pdf',dpi = 500,transparent=True)

In [ ]:
# each individual
# Font properties for scale bar
fontprops = fm.FontProperties(size=8)
# Add the scale bar and one 
for tp in ["Mock","D3PI","D7PI","D10PI","D14PI","D21PI"]:
    print(tp)
    # Add Padding
    global_min_x, global_max_x, global_min_y, global_max_y = float('inf'), float('-inf'), float('inf'), float('-inf')
    adata = adata_comb[adata_comb.obs['Sample']==tp].copy()
    coords = adata.obsm['spatial']
    global_min_x, global_max_x = min(global_min_x, coords[:, 0].min()), max(global_max_x, coords[:, 0].max())
    global_min_y, global_max_y = min(global_min_y, coords[:, 1].min()), max(global_max_y, coords[:, 1].max())

    # Optional padding
    padding = 50
    global_min_x -= padding
    global_max_x += padding
    global_min_y -= padding
    global_max_y += padding
    

    # Try add scale bar
    fig, ax = plt.subplots(1,1, figsize = (2,2))
    g = "num_clones_IG";i = "num_clones_TR"
    sc.pl.spatial(adata_comb_na[adata_comb_na.obs['Sample']==tp], color = g,spot_size=60,  ax = ax,show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_g,na_color="#E2E2E2")
    sc.pl.spatial(adata_comb_fillna[adata_comb_fillna.obs['Sample']==tp], color = g,spot_size=60,  ax = ax,show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_g,cmap=my_cmap2)
    sc.pl.spatial(adata_comb_fillna[adata_comb_fillna.obs['Sample']==tp], color = i,spot_size=60,  ax = ax,show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_i,cmap=my_cmap)
    # Set consistent axes limits
    ax.set_xlim(global_min_x, global_max_x)
    ax.set_ylim(global_max_y, global_min_y)
    ax.set_aspect('equal', adjustable='box')

    # add the scale bar
    scalebar = AnchoredSizeBar(ax.transData, 1.54*500, '', 'lower left',
                                pad=0.5, color='black', frameon=False, size_vertical=0.5,
                                fontproperties=fontprops)
    ax.add_artist(scalebar)
    # save
    plt.savefig('figures/sp_'+tp+"_num_clones.png",dpi = 500,transparent=True)
    plt.savefig('figures/sp_'+tp+"_num_clones.pdf",dpi = 500,transparent=True)

# Sup figures: For each AIR individually

In [ ]:
# need to deal with the color
# need to setup specific tips for the colorbar
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
import scanpy as sc
# Assuming 'adata_dict' contains your AnnData objects for different samples
adata_dict['D21PI2'] = adata_comb[adata_comb.obs['Sample']=="D21PI"]
samples_to_plot = ["Mock", "D3PI", "D7PI", "D10PI", "D14PI", "D21PI","D21PI2"]
genes_to_plot = ["num_clones_IGH","num_clones_IGK","num_clones_IGL","num_clones_TRB","num_clones_TRA"]  # Replace these with your gene names
sc.settings.set_figure_params(dpi=300, dpi_save= 300, facecolor='white', frameon=False, figsize=(1, 1), vector_friendly = False, transparent=True, format="pdf")
# Define color limits for each gene
vmax_vec = [4,7,3,4,2]
global_vmax = {}
for i in range(0,len(genes_to_plot)):
    g = genes_to_plot[i];v=vmax_vec[i]
    global_vmax[g] = v
cmap_vec = [my_cmap2,my_cmap2,my_cmap2,my_cmap,my_cmap]
# Define colormaps for each gene
cmp = {}
for i in range(0,len(genes_to_plot)):
    g = genes_to_plot[i];v=vmax_vec[i]
    cmp[g] = cmap_vec[i]


# Calculate global extents to ensure all plots have the same spatial limits
global_min_x, global_max_x, global_min_y, global_max_y = float('inf'), float('-inf'), float('inf'), float('-inf')
for sample in samples_to_plot:
    adata = adata_dict[sample]
    coords = adata.obsm['spatial']
    global_min_x, global_max_x = min(global_min_x, coords[:, 0].min()), max(global_max_x, coords[:, 0].max())
    global_min_y, global_max_y = min(global_min_y, coords[:, 1].min()), max(global_max_y, coords[:, 1].max())

# Optional padding
padding = 50
global_min_x -= padding
global_max_x += padding
global_min_y -= padding
global_max_y += padding

# Setup figure and axes
fig, axs = plt.subplots(len(samples_to_plot), len(genes_to_plot), figsize=(5, 7), squeeze=False)

    # Plot each gene for each sample with standardized spatial dimensions
fontprops = fm.FontProperties(size=5)
for i, sample_key in enumerate(samples_to_plot):
    adata = adata_dict[sample_key].copy()

    for j, gene in enumerate(genes_to_plot):
        ax = axs[i, j]
        sc.pl.spatial(adata, color=gene, ax=ax, show=False,
                    vmin=0,
                    vmax=global_vmax[gene],
                    cmap=cmp[gene], spot_size=60, frameon=False, colorbar_loc=None,  # No automatic colorbar
                    title='')  # Title only for top row

        # Set consistent axes limits
        ax.set_xlim(global_min_x, global_max_x)
        ax.set_ylim(global_max_y, global_min_y)
        ax.set_aspect('equal', adjustable='box')

        # Adding a scale bar to the first plot of each row
        if j == 0:
            scalebar = AnchoredSizeBar(ax.transData, 1.54*500, '', 'lower left',
                                    pad=0.1, color='black', frameon=False, size_vertical=1,
                                    fontproperties=fontprops)
            ax.add_artist(scalebar)

       # Add colorbar to the bottom of the last plot in each column
        if i == len(samples_to_plot) - 1:
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("bottom", size="10%", pad=0.05)  # Adjust 'size' to control thickness
            norm = plt.cm.colors.Normalize(vmin=0, vmax=global_vmax[gene])
            cbar = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmp[gene]), cax=cax, orientation='horizontal')
            cbar.set_label(gene)
            # Set specific tick positions for the colorbar
            tick_positions = np.linspace(0, global_vmax[gene], num =2)
            cbar.set_ticks(tick_positions)
            cbar.set_ticklabels([f"{t:.0f}" for t in tick_positions])

        # Remove any legend to clean up the plot
        if ax.get_legend() is not None:
            ax.get_legend().remove()

# Adjust subplot spacing and display the figure
plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig('figures/SP_num_clones.pdf', format='pdf', dpi=300)
plt.show()

# Sup Figures: Spatial Map on LR features

## MutFreq + MutCount + CDR3Len

### previous codes on plotting one by one

In [ ]:
# Font properties for scale bar
fontprops = fm.FontProperties(size=8)
# vdjs = ["IGH_MutFreq","IGK_MutFreq","IGL_MutFreq","IGH_MutCount","IGK_MutCount","IGL_MutCount","IGH_CDR3Len","IGK_CDR3Len","IGL_CDR3Len"]
vdjs = ["IGH_CDR3Len","IGK_CDR3Len","IGL_CDR3Len"]
for g in vdjs:
    print(g)
    adata = adata_comb.copy()
    if "IGH" in g:
        print(g)
        # convert NA in num_clones_IGH to the same as in g column
        adata.obs[g] = adata.obs[g].where(~adata.obs["num_clones_IGH"].isna(), other = np.nan)
        if "CDR3Len" in g:
            vmin = 10;vmax = 20
        else:
            vmin = 0
    elif "IGK" in g:
        # convert NA in num_clones_IGH to the same as in g column
        adata.obs[g] = adata.obs[g].where(~adata.obs["num_clones_IGK"].isna(), other = np.nan)
        if "CDR3Len" in g:
            vmin = 5;vmax = 15
        else:
            vmin = 0
    elif "IGL" in g:
        # convert NA in num_clones_IGH to the same as in g column
        adata.obs[g] = adata.obs[g].where(~adata.obs["num_clones_IGL"].isna(), other = np.nan)
        if "CDR3Len" in g:
            vmin = 5;vmax = 15
        else:
            vmin = 0
    # if "CDR3Len" in g:
    # get percentile
    vmax = np.nanpercentile(adata_comb.obs[g],99)
    sc.settings.set_figure_params(dpi=500, dpi_save= 500, fontsize=12, facecolor='white', frameon=False, figsize=(2, 2), vector_friendly = False, transparent=True, format="pdf")

    # Add the scale bar and one 
    for tp in ["Mock","D3PI","D7PI","D10PI","D14PI","D21PI"]:
        print(tp)
        # Add Padding
        global_min_x, global_max_x, global_min_y, global_max_y = float('inf'), float('-inf'), float('inf'), float('-inf')
        adata_t = adata[adata.obs['Sample']==tp].copy()
        coords = adata_t.obsm['spatial']
        global_min_x, global_max_x = min(global_min_x, coords[:, 0].min()), max(global_max_x, coords[:, 0].max())
        global_min_y, global_max_y = min(global_min_y, coords[:, 1].min()), max(global_max_y, coords[:, 1].max())
        # Optional padding
        padding = 50
        global_min_x -= padding
        global_max_x += padding
        global_min_y -= padding
        global_max_y += padding
        

        # Try add scale bar
        fig, ax = plt.subplots(1,1, figsize = (2,2))
        sc.pl.spatial(adata_t, color = g,spot_size=60,  ax = ax,show=False,title="",colorbar_loc=None, vmin=vmin,vmax=vmax)
        # g = "num_clones_IG";i = "num_clones_TR"
        # sc.pl.spatial(adata_comb_na[adata_comb_na.obs['Sample']==tp], color = g,spot_size=60,  ax = ax,show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_g,na_color="#E2E2E2")
        # sc.pl.spatial(adata_comb_fillna[adata_comb_fillna.obs['Sample']==tp], color = g,spot_size=60,  ax = ax,show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_g,cmap=my_cmap2)
        # sc.pl.spatial(adata_comb_fillna[adata_comb_fillna.obs['Sample']==tp], color = i,spot_size=60,  ax = ax,show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_i,cmap=my_cmap)
        # Set consistent axes limits
        ax.set_xlim(global_min_x, global_max_x)
        ax.set_ylim(global_max_y, global_min_y)
        ax.set_aspect('equal', adjustable='box')

        # add the scale bar
        scalebar = AnchoredSizeBar(ax.transData, 1.54*500, '', 'lower left',
                                    pad=0.5, color='black', frameon=False, size_vertical=0.5,
                                    fontproperties=fontprops)
        ax.add_artist(scalebar)
        # save
        plt.savefig('figures/sp_'+tp+"_"+g+".png",dpi = 500,transparent=True)
        plt.savefig('figures/sp_'+tp+"_"+g+".pdf",dpi = 500,transparent=True)
        # print(vmax)
    # for legend
    sc.pl.spatial(adata_t, color = g,spot_size=60,show=False,title="", vmin=vmin,vmax=vmax,save="sp_"+g+"_Legend.pdf")

### New codes on ploting together; but the legend scale bar looks weird

In [ ]:
# cmap for virids
from matplotlib import colors
# Assuming 'adata_dict' contains your AnnData objects for different samples
adata_dict['D21PI2'] = adata_comb[adata_comb.obs['Sample']=="D21PI"]
samples_to_plot = ["Mock", "D3PI", "D7PI", "D10PI", "D14PI", "D21PI","D21PI2"]
genes_to_plot = ['IGH_MutFreq','IGK_MutFreq', 'IGL_MutFreq']  # Replace these with your gene names
sc.settings.set_figure_params(dpi=300, dpi_save= 300, facecolor='white', frameon=False, figsize=(1, 1), vector_friendly = False, transparent=True, format="pdf")
# Define color limits for each gene
vmax_vec = [0.05,0.05,0.05]
global_vmax = {}
for i in range(0,len(genes_to_plot)):
    g = genes_to_plot[i];v=vmax_vec[i]
    global_vmax[g] = v
cmap_vec = ["viridis","viridis","viridis"]
# Define colormaps for each gene
cmp = {}
for i in range(0,len(genes_to_plot)):
    g = genes_to_plot[i];v=vmax_vec[i]
    cmp[g] = cmap_vec[i]


# Calculate global extents to ensure all plots have the same spatial limits
global_min_x, global_max_x, global_min_y, global_max_y = float('inf'), float('-inf'), float('inf'), float('-inf')
for sample in samples_to_plot:
    adata = adata_dict[sample]
    coords = adata.obsm['spatial']
    global_min_x, global_max_x = min(global_min_x, coords[:, 0].min()), max(global_max_x, coords[:, 0].max())
    global_min_y, global_max_y = min(global_min_y, coords[:, 1].min()), max(global_max_y, coords[:, 1].max())

# Optional padding
padding = 50
global_min_x -= padding
global_max_x += padding
global_min_y -= padding
global_max_y += padding

# Setup figure and axes
fig, axs = plt.subplots(len(samples_to_plot), len(genes_to_plot), figsize=(5, 7), squeeze=False)

    # Plot each gene for each sample with standardized spatial dimensions
fontprops = fm.FontProperties(size=5)
for i, sample_key in enumerate(samples_to_plot):
    adata = adata_dict[sample_key].copy()
    adata.obs[genes_to_plot] = adata.obs[genes_to_plot].replace(np.nan, 0)
    for j, gene in enumerate(genes_to_plot):
        ax = axs[i, j]
        sc.pl.spatial(adata, color=gene, ax=ax, show=False,
                    vmin=0,
                    vmax=global_vmax[gene],
                    cmap=cmp[gene], spot_size=60, frameon=False, colorbar_loc=None,  # No automatic colorbar
                    title='')  # Title only for top row

        # Set consistent axes limits
        ax.set_xlim(global_min_x, global_max_x)
        ax.set_ylim(global_max_y, global_min_y)
        ax.set_aspect('equal', adjustable='box')

        # Adding a scale bar to the first plot of each row
        if j == 0:
            scalebar = AnchoredSizeBar(ax.transData, 1.54*500, '', 'lower left',
                                    pad=0.1, color='black', frameon=False, size_vertical=1,
                                    fontproperties=fontprops)
            ax.add_artist(scalebar)

       # Add colorbar to the bottom of the last plot in each column
        if i == len(samples_to_plot) - 1:
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("bottom", size="10%", pad=0.05)  # Adjust 'size' to control thickness
            norm = plt.cm.colors.Normalize(vmin=0, vmax=global_vmax[gene])
            cbar = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmp[gene]), cax=cax, orientation='horizontal')
            cbar.set_label(gene)
            # Set specific tick positions for the colorbar
            tick_positions = np.linspace(0, global_vmax[gene], num =2)
            cbar.set_ticks(tick_positions)
            cbar.set_ticklabels([f"{t:.0f}" for t in tick_positions])

        # Remove any legend to clean up the plot
        if ax.get_legend() is not None:
            ax.get_legend().remove()

# Adjust subplot spacing and display the figure
plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig('figures/SP_IG_MutFreq.pdf', format='pdf', dpi=300)
plt.show()

### TCR MutFreq

In [ ]:
samples_to_plot = ["Mock", "D3PI", "D7PI", "D10PI", "D14PI", "D21PI","D21PI2"]
genes_to_plot = ['TRB_MutFreq','TRA_MutFreq']  # Replace these with your gene names
sc.settings.set_figure_params(dpi=300, dpi_save= 300, facecolor='white', frameon=False, figsize=(1, 1), vector_friendly = False, transparent=True, format="pdf")
# Define color limits for each gene
vmax_vec = [0.05,0.05]
global_vmax = {}
for i in range(0,len(genes_to_plot)):
    g = genes_to_plot[i];v=vmax_vec[i]
    global_vmax[g] = v
cmap_vec = ["viridis","viridis"]
# Define colormaps for each gene
cmp = {}
for i in range(0,len(genes_to_plot)):
    g = genes_to_plot[i];v=vmax_vec[i]
    cmp[g] = cmap_vec[i]


# Calculate global extents to ensure all plots have the same spatial limits
global_min_x, global_max_x, global_min_y, global_max_y = float('inf'), float('-inf'), float('inf'), float('-inf')
for sample in samples_to_plot:
    adata = adata_dict[sample]
    coords = adata.obsm['spatial']
    global_min_x, global_max_x = min(global_min_x, coords[:, 0].min()), max(global_max_x, coords[:, 0].max())
    global_min_y, global_max_y = min(global_min_y, coords[:, 1].min()), max(global_max_y, coords[:, 1].max())

# Optional padding
padding = 50
global_min_x -= padding
global_max_x += padding
global_min_y -= padding
global_max_y += padding

# Setup figure and axes
fig, axs = plt.subplots(len(samples_to_plot), len(genes_to_plot), figsize=(5, 7), squeeze=False)

    # Plot each gene for each sample with standardized spatial dimensions
fontprops = fm.FontProperties(size=5)
for i, sample_key in enumerate(samples_to_plot):
    adata = adata_dict[sample_key].copy()
    adata.obs[genes_to_plot] = adata.obs[genes_to_plot].replace(np.nan, 0)
    for j, gene in enumerate(genes_to_plot):
        ax = axs[i, j]
        sc.pl.spatial(adata, color=gene, ax=ax, show=False,
                    vmin=0,
                    vmax=global_vmax[gene],
                    cmap=cmp[gene], spot_size=60, frameon=False, colorbar_loc=None,  # No automatic colorbar
                    title='')  # Title only for top row

        # Set consistent axes limits
        ax.set_xlim(global_min_x, global_max_x)
        ax.set_ylim(global_max_y, global_min_y)
        ax.set_aspect('equal', adjustable='box')

        # Adding a scale bar to the first plot of each row
        if j == 0:
            scalebar = AnchoredSizeBar(ax.transData, 1.54*500, '', 'lower left',
                                    pad=0.1, color='black', frameon=False, size_vertical=1,
                                    fontproperties=fontprops)
            ax.add_artist(scalebar)

       # Add colorbar to the bottom of the last plot in each column
        if i == len(samples_to_plot) - 1:
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("bottom", size="10%", pad=0.05)  # Adjust 'size' to control thickness
            norm = plt.cm.colors.Normalize(vmin=0, vmax=global_vmax[gene])
            cbar = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmp[gene]), cax=cax, orientation='horizontal')
            cbar.set_label(gene)
            # Set specific tick positions for the colorbar
            tick_positions = np.linspace(0, global_vmax[gene], num =2)
            cbar.set_ticks(tick_positions)
            cbar.set_ticklabels([f"{t:.0f}" for t in tick_positions])

        # Remove any legend to clean up the plot
        if ax.get_legend() is not None:
            ax.get_legend().remove()

# Adjust subplot spacing and display the figure
plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig('figures/SP_TR_MutFreq.pdf', format='pdf', dpi=300)
plt.show()

### Number of Mutations

In [ ]:
# Assuming 'adata_dict' contains your AnnData objects for different samples
# adata_dict['D21PI2'] = adata_comb[adata_comb.obs['Sample']=="D21PI"]
samples_to_plot = ["Mock", "D3PI", "D7PI", "D10PI", "D14PI", "D21PI","D21PI2"]
genes_to_plot = ['IGH_MutCount','IGK_MutCount', 'IGL_MutCount']  # Replace these with your gene names
sc.settings.set_figure_params(dpi=300, dpi_save= 300, facecolor='white', frameon=False, figsize=(1, 1), vector_friendly = False, transparent=True, format="pdf")
# Define color limits for each gene
vmax_vec = [5,5,5]
global_vmax = {}
for i in range(0,len(genes_to_plot)):
    g = genes_to_plot[i];v=vmax_vec[i]
    global_vmax[g] = v
cmap_vec = ["viridis","viridis","viridis"]
# Define colormaps for each gene
cmp = {}
for i in range(0,len(genes_to_plot)):
    g = genes_to_plot[i];v=vmax_vec[i]
    cmp[g] = cmap_vec[i]


# Calculate global extents to ensure all plots have the same spatial limits
global_min_x, global_max_x, global_min_y, global_max_y = float('inf'), float('-inf'), float('inf'), float('-inf')
for sample in samples_to_plot:
    adata = adata_dict[sample]
    coords = adata.obsm['spatial']
    global_min_x, global_max_x = min(global_min_x, coords[:, 0].min()), max(global_max_x, coords[:, 0].max())
    global_min_y, global_max_y = min(global_min_y, coords[:, 1].min()), max(global_max_y, coords[:, 1].max())

# Optional padding
padding = 50
global_min_x -= padding
global_max_x += padding
global_min_y -= padding
global_max_y += padding

# Setup figure and axes
fig, axs = plt.subplots(len(samples_to_plot), len(genes_to_plot), figsize=(5, 7), squeeze=False)

    # Plot each gene for each sample with standardized spatial dimensions
fontprops = fm.FontProperties(size=5)
for i, sample_key in enumerate(samples_to_plot):
    adata = adata_dict[sample_key].copy()
    adata.obs[genes_to_plot] = adata.obs[genes_to_plot].replace(np.nan, 0)
    for j, gene in enumerate(genes_to_plot):
        ax = axs[i, j]
        sc.pl.spatial(adata, color=gene, ax=ax, show=False,
                    vmin=0,
                    vmax=global_vmax[gene],
                    cmap=cmp[gene], spot_size=60, frameon=False, colorbar_loc=None,  # No automatic colorbar
                    title='')  # Title only for top row

        # Set consistent axes limits
        ax.set_xlim(global_min_x, global_max_x)
        ax.set_ylim(global_max_y, global_min_y)
        ax.set_aspect('equal', adjustable='box')

        # Adding a scale bar to the first plot of each row
        if j == 0:
            scalebar = AnchoredSizeBar(ax.transData, 1.54*500, '', 'lower left',
                                    pad=0.1, color='black', frameon=False, size_vertical=1,
                                    fontproperties=fontprops)
            ax.add_artist(scalebar)

       # Add colorbar to the bottom of the last plot in each column
        if i == len(samples_to_plot) - 1:
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("bottom", size="10%", pad=0.05)  # Adjust 'size' to control thickness
            norm = plt.cm.colors.Normalize(vmin=0, vmax=global_vmax[gene])
            cbar = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmp[gene]), cax=cax, orientation='horizontal')
            cbar.set_label(gene)
            # Set specific tick positions for the colorbar
            tick_positions = np.linspace(0, global_vmax[gene], num =2)
            cbar.set_ticks(tick_positions)
            cbar.set_ticklabels([f"{t:.0f}" for t in tick_positions])

        # Remove any legend to clean up the plot
        if ax.get_legend() is not None:
            ax.get_legend().remove()

# Adjust subplot spacing and display the figure
plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig('figures/SP_IG_MutCount.pdf', format='pdf', dpi=300)
plt.show()

# Supp Fig: Spatial map on pair-receptor beads

In [ ]:
adata_dict['D21PI2'] = adata_comb[adata_comb.obs['Sample']=="D21PI"]
samples_to_plot = ["Mock", "D3PI", "D7PI", "D10PI", "D14PI", "D21PI","D21PI2"]
genes_to_plot = ["num_clones_IGpair_HK","num_clones_IGpair_HL","num_clones_TRpair","num_clones_BTpair_HB"]  # Replace these with your gene names
sc.settings.set_figure_params(dpi=300, dpi_save= 300, facecolor='white', frameon=False, figsize=(1, 1), vector_friendly = False, transparent=True, format="pdf")
# Define color limits for each gene
vmax_vec = [1,1,1,1]
global_vmax = {}
for i in range(0,len(genes_to_plot)):
    g = genes_to_plot[i];v=vmax_vec[i]
    global_vmax[g] = v
cmap_vec = ["Reds","Reds","Reds","Reds"]
# Define colormaps for each gene
cmp = {}
for i in range(0,len(genes_to_plot)):
    g = genes_to_plot[i];v=vmax_vec[i]
    cmp[g] = cmap_vec[i]


# Calculate global extents to ensure all plots have the same spatial limits
global_min_x, global_max_x, global_min_y, global_max_y = float('inf'), float('-inf'), float('inf'), float('-inf')
for sample in samples_to_plot:
    adata = adata_dict[sample]
    coords = adata.obsm['spatial']
    global_min_x, global_max_x = min(global_min_x, coords[:, 0].min()), max(global_max_x, coords[:, 0].max())
    global_min_y, global_max_y = min(global_min_y, coords[:, 1].min()), max(global_max_y, coords[:, 1].max())

# Optional padding
padding = 50
global_min_x -= padding
global_max_x += padding
global_min_y -= padding
global_max_y += padding

# Setup figure and axes
fig, axs = plt.subplots(len(samples_to_plot), len(genes_to_plot), figsize=(4, 7), squeeze=False)

    # Plot each gene for each sample with standardized spatial dimensions
fontprops = fm.FontProperties(size=5)
for i, sample_key in enumerate(samples_to_plot):
    adata = adata_dict[sample_key].copy()

    for j, gene in enumerate(genes_to_plot):
        ax = axs[i, j]
        sc.pl.spatial(adata, color=gene, ax=ax, show=False,
                    vmin=0,
                    vmax=global_vmax[gene],
                    cmap=cmp[gene], spot_size=60, frameon=False, colorbar_loc=None,  # No automatic colorbar
                    title='')  # Title only for top row

        # Set consistent axes limits
        ax.set_xlim(global_min_x, global_max_x)
        ax.set_ylim(global_max_y, global_min_y)
        ax.set_aspect('equal', adjustable='box')

        # Adding a scale bar to the first plot of each row
        if j == 0:
            scalebar = AnchoredSizeBar(ax.transData, 1.54*500, '', 'lower left',
                                    pad=0.1, color='black', frameon=False, size_vertical=1,
                                    fontproperties=fontprops)
            ax.add_artist(scalebar)

       # Add colorbar to the bottom of the last plot in each column
        if i == len(samples_to_plot) - 1:
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("bottom", size="10%", pad=0.05)  # Adjust 'size' to control thickness
            norm = plt.cm.colors.Normalize(vmin=0, vmax=global_vmax[gene])
            cbar = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmp[gene]), cax=cax, orientation='horizontal')
            cbar.set_label(gene)
            # Set specific tick positions for the colorbar
            tick_positions = np.linspace(0, global_vmax[gene], num =3)
            cbar.set_ticks(tick_positions)
            cbar.set_ticklabels([f"{t:.0f}" for t in tick_positions])

        # Remove any legend to clean up the plot
        if ax.get_legend() is not None:
            ax.get_legend().remove()

# Adjust subplot spacing and display the figure
plt.subplots_adjust(wspace=0, hspace=0.05)
plt.savefig('figures/SP_pair_clones.pdf', format='pdf', dpi=300)
plt.show()

# Supp Fig: Spatial map on Entropy

In [ ]:
# add the entropy levels calculated from pairHL in R
entropy_df = pd.read_csv("/fs/cbsuvlaminck5/workdir/sj657/lymph_node/for_public/output_files/meta_allTP_pairHL_entropy.tsv",sep = ",")
entropy_df
adata_comb.obs = pd.merge(adata_comb.obs, entropy_df, how = "left", left_index = True, right_on = "Index")

### Old code

In [ ]:
fontprops = fm.FontProperties(size=8)
# vdjs = ["IGH_MutFreq","IGK_MutFreq","IGL_MutFreq","IGH_MutCount","IGK_MutCount","IGL_MutCount","IGH_CDR3Len","IGK_CDR3Len","IGL_CDR3Len"]
vdjs = ["entropy"]
for g in vdjs:
    print(g)
    adata = adata_comb.copy()
    vmin=0;vmax=5
    adata.obs.index = adata.obs['barcode']

    print(vmax)
    sc.settings.set_figure_params(dpi=500, dpi_save= 500, fontsize=12, facecolor='white', frameon=False, figsize=(2, 2), vector_friendly = False, transparent=True, format="pdf")

    # Add the scale bar and one 
    for tp in ["Mock","D3PI","D7PI","D10PI","D14PI","D21PI"]:
        print(tp)
        # Add Padding
        global_min_x, global_max_x, global_min_y, global_max_y = float('inf'), float('-inf'), float('inf'), float('-inf')
        adata_t = adata[adata.obs['Sample']==tp].copy()
        coords = adata_t.obsm['spatial']
        global_min_x, global_max_x = min(global_min_x, coords[:, 0].min()), max(global_max_x, coords[:, 0].max())
        global_min_y, global_max_y = min(global_min_y, coords[:, 1].min()), max(global_max_y, coords[:, 1].max())
        # Optional padding
        padding = 50
        global_min_x -= padding
        global_max_x += padding
        global_min_y -= padding
        global_max_y += padding
        

        # Try add scale bar
        fig, ax = plt.subplots(1,1, figsize = (2,2))
        sc.pl.spatial(adata_t, color = g,spot_size=60,  ax = ax,show=False,title="",colorbar_loc=None, vmin=vmin,vmax=vmax)
        # Set consistent axes limits
        ax.set_xlim(global_min_x, global_max_x)
        ax.set_ylim(global_max_y, global_min_y)
        ax.set_aspect('equal', adjustable='box')

        # add the scale bar
        scalebar = AnchoredSizeBar(ax.transData, 1.54*500, '', 'lower left',
                                    pad=0.5, color='black', frameon=False, size_vertical=0.5,
                                    fontproperties=fontprops)
        ax.add_artist(scalebar)
        # save
        plt.savefig('figures/sp_'+tp+"_"+g+".png",dpi = 500,transparent=True)
        plt.savefig('figures/sp_'+tp+"_"+g+".pdf",dpi = 500,transparent=True)
        # print(vmax)
    # for legend
    sc.settings.set_figure_params(dpi=500, dpi_save= 500, fontsize=6, facecolor='white', frameon=False, figsize=(2, 2), vector_friendly = False, transparent=True, format="pdf")
    sc.pl.spatial(adata_t, color = g,spot_size=60,show=False,title="", vmin=vmin,vmax=vmax,save="sp_"+g+"_Legend.pdf")

### New code:

In [ ]:
# Plotting in a column with allTP
# need to deal with the color
# need to setup specific tips for the colorbar
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
import scanpy as sc
# Assuming 'adata_dict' contains your AnnData objects for different samples
adata_dict['D21PI2'] = adata_comb[adata_comb.obs['Sample']=="D21PI"]
samples_to_plot = ["Mock", "D3PI", "D7PI", "D10PI", "D14PI", "D21PI","D21PI2"]
genes_to_plot = ["entropy"]  # Replace these with your gene names
sc.settings.set_figure_params(dpi=300, dpi_save= 300, facecolor='white', frameon=False, figsize=(1, 1), vector_friendly = False, transparent=True, format="pdf")
# Define color limits for each gene
vmax_vec = [5]
global_vmax = {}
for i in range(0,len(genes_to_plot)):
    g = genes_to_plot[i];v=vmax_vec[i]
    global_vmax[g] = v
cmap_vec = ["viridis"]
# Define colormaps for each gene
cmp = {}
for i in range(0,len(genes_to_plot)):
    g = genes_to_plot[i];v=vmax_vec[i]
    cmp[g] = cmap_vec[i]


# Calculate global extents to ensure all plots have the same spatial limits
global_min_x, global_max_x, global_min_y, global_max_y = float('inf'), float('-inf'), float('inf'), float('-inf')
for sample in samples_to_plot:
    adata = adata_dict[sample]
    coords = adata.obsm['spatial']
    global_min_x, global_max_x = min(global_min_x, coords[:, 0].min()), max(global_max_x, coords[:, 0].max())
    global_min_y, global_max_y = min(global_min_y, coords[:, 1].min()), max(global_max_y, coords[:, 1].max())

# Optional padding
padding = 50
global_min_x -= padding
global_max_x += padding
global_min_y -= padding
global_max_y += padding

# Setup figure and axes
fig, axs = plt.subplots(len(samples_to_plot), len(genes_to_plot), figsize=(4, 7), squeeze=False)

    # Plot each gene for each sample with standardized spatial dimensions
fontprops = fm.FontProperties(size=5)
for i, sample_key in enumerate(samples_to_plot):
    adata = adata_dict[sample_key].copy()

    for j, gene in enumerate(genes_to_plot):
        ax = axs[i, j]
        sc.pl.spatial(adata, color=gene, ax=ax, show=False,
                    vmin=0,
                    vmax=global_vmax[gene],
                    cmap=cmp[gene], spot_size=60, frameon=False, colorbar_loc=None,  # No automatic colorbar
                    title='')  # Title only for top row

        # Set consistent axes limits
        ax.set_xlim(global_min_x, global_max_x)
        ax.set_ylim(global_max_y, global_min_y)
        ax.set_aspect('equal', adjustable='box')

        # Adding a scale bar to the first plot of each row
        if j == 0:
            scalebar = AnchoredSizeBar(ax.transData, 1.54*500, '', 'lower left',
                                    pad=0.1, color='black', frameon=False, size_vertical=1,
                                    fontproperties=fontprops)
            ax.add_artist(scalebar)

       # Add colorbar to the bottom of the last plot in each column
        if i == len(samples_to_plot) - 1:
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("bottom", size="10%", pad=0.05)  # Adjust 'size' to control thickness
            norm = plt.cm.colors.Normalize(vmin=0, vmax=global_vmax[gene])
            cbar = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmp[gene]), cax=cax, orientation='horizontal')
            cbar.set_label(gene)
            # Set specific tick positions for the colorbar
            tick_positions = np.linspace(0, global_vmax[gene], num=3)
            cbar.set_ticks(tick_positions)
            cbar.set_ticklabels([f"{t:.0f}" for t in tick_positions])

        # Remove any legend to clean up the plot
        if ax.get_legend() is not None:
            ax.get_legend().remove()

# Adjust subplot spacing and display the figure
plt.subplots_adjust(wspace=0, hspace=0.05)
plt.savefig('figures/SP_pair_entropy.pdf', format='pdf', dpi=300)
plt.show()

# Supp Fig: Spatial map of CD4 CD8 peripheral or central

In [ ]:
t_color_dict = {"CD4 T cells": "#00A175",
                "CD8 T cells": "#7A7B1B"}

# Font properties for scale bar
fontprops = fm.FontProperties(size=8)
sc.settings.set_figure_params(dpi=500, dpi_save= 500, fontsize=2, facecolor='white', frameon=False, figsize=(3, 3), vector_friendly = False, transparent=True, format="pdf")
for tp in ["Mock","D3PI","D7PI","D10PI","D14PI","D21PI"]:
    print(tp)
    # Add Padding
    global_min_x, global_max_x, global_min_y, global_max_y = float('inf'), float('-inf'), float('inf'), float('-inf')
    adata = adata_comb[adata_comb.obs['Sample']==tp].copy()
    coords = adata.obsm['spatial']
    global_min_x, global_max_x = min(global_min_x, coords[:, 0].min()), max(global_max_x, coords[:, 0].max())
    global_min_y, global_max_y = min(global_min_y, coords[:, 1].min()), max(global_max_y, coords[:, 1].max())

    # Optional padding
    padding = 50
    global_min_x -= padding
    global_max_x += padding
    global_min_y -= padding
    global_max_y += padding
    

    # Try add scale bar
    fig, ax = plt.subplots(1,1, figsize = (3,3))
    g = "CD4_CD8_type"
    sc.pl.spatial(adata, color = g,title='',spot_size=50,show=False,legend_loc=None, ax = ax)
    # Set consistent axes limits
    ax.set_xlim(global_min_x, global_max_x)
    ax.set_ylim(global_max_y, global_min_y)
    ax.set_aspect('equal', adjustable='box')

    # add the scale bar
    scalebar = AnchoredSizeBar(ax.transData, 1.54*500, '', 'lower left',
                                pad=0.5, color='black', frameon=False, size_vertical=0.5,
                                fontproperties=fontprops)
    ax.add_artist(scalebar)
    # save
    plt.savefig('figures/sp_'+tp+"_"+g+'.png',dpi = 500,transparent=True)
    plt.savefig('figures/sp_'+tp+"_"+g+'.pdf',dpi = 500,transparent=True)
# one for legend
fig, ax = plt.subplots(1,1, figsize = (3,3))
g = "CD4_CD8_type"
sc.settings.set_figure_params(dpi=500, dpi_save= 500, fontsize=6, facecolor='white', frameon=False, figsize=(3, 3), vector_friendly = False, transparent=True, format="pdf")
sc.pl.spatial(adata, color = g,title='',spot_size=35,show=False, ax = ax)
# save
plt.savefig('figures/sp_'+tp+"_"+g+'Legend.png',dpi = 500,transparent=True)
plt.savefig('figures/sp_'+tp+"_"+g+'Legend.pdf',dpi = 500,transparent=True)

# Supp Fig: Matrix plot on cell type gene markers

In [ ]:
# from adata_comb
adata_scale = adata_comb.copy()
sc.pp.normalize_total(adata_scale, inplace=True)
sc.pp.log1p(adata_scale)
sc.pp.regress_out(adata_scale, ["total_counts"])
sc.pp.scale(adata_scale, max_value=10)
# adata_comb.X.sum(1)
# adata_scale.write_h5ad("../../../output_files/objects/adata_comb_allTP_whole_meta_scale.h5ad")


In [4]:
# Using the scale data
adata_scale = sc.read_h5ad("../../../output_files/objects/adata_comb_allTP_whole_meta_scale.h5ad")
adata_scale.obs

,barcode,x,y,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,Sample,Index,S_score,G2M_score,phase,Celltypes_c2lLN,Broad_celltypes,Refine_celltypes,CD4_CD8_type,Bcell_subtype_c2lHTA,Region,num_ONT_reads,num_ONT_umi,num_align_reads,num_align_umi,num_TRBC_reads,num_TRBC_umi,num_TRAC_reads,...,IGH_CDR3Len,IGK_CDR3Len,IGL_CDR3Len,TRA_CDR3Len,TRB_CDR3Len,num_IGH_umi,num_clones_IGpair_HK,num_clones_IGpair_HL,num_clones_IGpair,num_clones_BTpair_HB,IGHM_M,IGHD_M,IGHM_S,IGHG1_S,IGHG2B_S,IGHG2C_S,IGHG3_S,IGHD_S,IGHG2B_M,IGHG1_M,IGHG3_M,IGHA_S,IGHE_S,IGHA_M,ge_Aicda
Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TCACTGCCGGTGGC_Mock,TCACTGCCGGTGGC,4906.9,2672.2,493,1067.0,47.0,4.404873,Mock,TCACTGCCGGTGGC_Mock,-0.091706,-0.029953,G1,B_cells,B cells,B_Naive,NaN,B_Naive,Outer Cortex,18.0,14.0,4.0,3.0,0.0,0.0,0.0,...,18.0,11.0,0.0,0.0,0.0,1.0,2.0,NaN,NaN,NaN,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
AGACTCTACGCGGG_Mock,AGACTCTACGCGGG,2439.5,3825.3,425,643.0,16.0,2.488336,Mock,AGACTCTACGCGGG_Mock,0.051156,-0.029248,S,Adipocytes,Stroma,Adipocytes,NaN,NaN,Outer Cortex,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,10.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
GGCTGCTGCCCAGC_Mock,GGCTGCTGCCCAGC,3927.0,2598.0,596,1157.0,34.0,2.938634,Mock,GGCTGCTGCCCAGC_Mock,-0.037062,-0.098831,G1,B_cells,B cells,B_Naive,NaN,B_Naive,Outer Cortex,13.0,11.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,11.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
TGTCTTATTGACTC_Mock,TGTCTTATTGACTC,4389.0,1892.0,662,1269.0,24.0,1.891253,Mock,TGTCTTATTGACTC_Mock,-0.039739,-0.178353,G1,T_CD4+,T cells,T_CD4,CD4 T cells,NaN,Inner Cortex,9.0,9.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,15.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
CTAATGTTACTAGT_Mock,CTAATGTTACTAGT,3971.0,1723.8,236,317.0,3.0,0.946372,Mock,CTAATGTTACTAGT_Mock,-0.176514,-0.150827,G1,Endo,Stroma,Endo,NaN,NaN,Medulla,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TACCTTCCTTCTCA_D21PI,TACCTTCCTTCTCA,780.2,3969.8,253,373.0,5.0,1.340483,D21PI,TACCTTCCTTCTCA_D21PI,-0.099070,-0.016102,G1,B_cells,B cells,B_Naive,NaN,B_Naive,Capsule,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
TAGACAAAATACAT_D21PI,TAGACAAAATACAT,2619.0,3229.7,397,633.0,13.0,2.053713,D21PI,TAGACAAAATACAT_D21PI,0.035630,-0.081908,S,T_CD4+,T cells,T_CD8,CD8 T cells,NaN,Inner Cortex,2.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
GTCAGATTCGTTGG_D21PI,GTCAGATTCGTTGG,2502.2,4370.5,575,974.0,12.0,1.232033,D21PI,GTCAGATTCGTTGG_D21PI,-0.142306,0.017438,G2M,B_Cycling,B cell follicles,B_Act_Naive,NaN,B_Act_Naive,Germinal Center,24.0,24.0,6.0,6.0,0.0,0.0,0.0,...,17.0,11.0,11.0,0.0,0.0,1.0,2.0,2.0,4.0,NaN,2.0,NaN,NaN,2.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [5]:
adata_scale.X.sum(1)

array([-373.6071  ,  159.70529 , -229.66473 , ...,    2.734747,
       -472.54504 ,   94.717606], dtype=float32)

/home/sj657/miniconda3/envs/c2l_env/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


array([-373.60785  ,  159.70482  , -229.66434  , ...,    2.7345936,
       -472.54578  ,   94.71733  ], dtype=float32)

In [ ]:
marker_dict = {
'T': ['Cd3d', 'Cd4','Cd8a'],
'NK':['Nkg7'],
# 'Tfh_GC':['Bcl6'],
'B': ['Cd79a','Cd79b','Aicda','Bcl6','Xbp1','Cd74','Ighd','Ighm'],
'FDC':['Cr2'],
'DC':['Actb'],
'Macro_Mono':['Apoe','Lyz2'],
'Endo':['Selenop'],
'Adipo':['Cfd','Fabp4']
# ''
}
marker_list = [value for key,values in marker_dict.items() for value in values]
g = "Refine_celltypes"
sc.settings.set_figure_params(dpi=300, dpi_save= 300, fontsize=15, facecolor='white', frameon=False, figsize=(8, 8), vector_friendly = False, transparent=True, format="pdf")
sc.pl.matrixplot(adata_scale, marker_list, groupby=g,save="MatPlot_tmp_markers_Refinecp_TP.pdf",dendrogram=False,swap_axes=True,cmap="Blues",
                 categories_order=['T_CD4','T_CD4_Cycling','T_CD8','T_CD8_Cycling','NK_ILC','B_Naive','B_Act_Naive','B_GC','B_Mem','B_plasma','FDC','DCs','Macrophage','Monocytes','VSMC','Endo','Adipocytes'],vmin=0,vmax=0.5)

# Supp Fig: Other gene expression

In [4]:
adata_norm = adata_comb.copy()
sc.pp.normalize_total(adata_norm, inplace=True)
sc.pp.log1p(adata_norm)

In [ ]:
# Maybe use normalized value, makes more sense
sc.settings.set_figure_params(dpi=500, dpi_save= 500, fontsize=8, facecolor='white', frameon=False, figsize=(2, 2), vector_friendly = False, transparent=True, format="pdf")
fontprops = fm.FontProperties(size=8)
genes = ["Trac","Trbc2","Ighg1","Iglc1","Igkc"]
for g in genes:
    for tp in ["D14PI"]:
        # vmax = np.percentile(adata_norm[:,g].X,99)
        # print(vmax)
        print(tp)
        # Add Padding
        global_min_x, global_max_x, global_min_y, global_max_y = float('inf'), float('-inf'), float('inf'), float('-inf')
        adata = adata_norm[adata_norm.obs['Sample']==tp].copy()
        coords = adata.obsm['spatial']
        global_min_x, global_max_x = min(global_min_x, coords[:, 0].min()), max(global_max_x, coords[:, 0].max())
        global_min_y, global_max_y = min(global_min_y, coords[:, 1].min()), max(global_max_y, coords[:, 1].max())
    
        # Optional padding
        padding = 50
        global_min_x -= padding
        global_max_x += padding
        global_min_y -= padding
        global_max_y += padding
        
    
        # Try add scale bar
        fig, ax = plt.subplots(1,1, figsize = (2,2))
        # g = "Ifng"
        sc.pl.spatial(adata_norm[adata_norm.obs['Sample']==tp],color = g,spot_size=60,  ax = ax,show=False,title= "",colorbar_loc=None, vmin=0,vmax='p99')
        # sc.pl.spatial(adata_comb_fillna[adata_comb_fillna.obs['Sample']==tp], color = g,spot_size=60,  ax = ax,show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_g,cmap=my_cmap2)
        # sc.pl.spatial(adata_comb_fillna[adata_comb_fillna.obs['Sample']==tp], color = i,spot_size=60,  ax = ax,show=False,title="",colorbar_loc=None, vmin=0,vmax=vmax_i,cmap=my_cmap)
        # Set consistent axes limits
        ax.set_xlim(global_min_x, global_max_x)
        ax.set_ylim(global_max_y, global_min_y)
        ax.set_aspect('equal', adjustable='box')
    
        # # add the scale bar
        # scalebar = AnchoredSizeBar(ax.transData, 1.54*500, '', 'lower left',
        #                             pad=0.5, color='black', frameon=False, size_vertical=0.5,
        #                             fontproperties=fontprops)
        # ax.add_artist(scalebar)
        # # save
        plt.savefig('figures/sp_'+g+"_norm_gene_exp_"+tp+".png",dpi = 500,transparent=True)
        plt.savefig('figures/sp_'+g+"_norm_gene_exp_"+tp+".pdf",dpi = 500,transparent=True)

### Aicda expression 

In [13]:
adata_norm = adata_comb.copy()
sc.pp.normalize_total(adata_norm, inplace=True)
sc.pp.log1p(adata_norm)

In [19]:
# add to the meta table
g = "Aicda"
adata_norm.obs["ge_Aicda"] = adata_norm[:,g].X.toarray()

In [20]:
adata_norm.obs['ge_Aicda'].describe()

count    133826.000000
mean          0.019521
std           0.115329
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           2.204793
Name: ge_Aicda, dtype: float64